In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoModel, AutoTokenizer
from swarm.optimizer.edge_optimizer.edge_network import EdgeNetwork
from swarm.optimizer.edge_optimizer.parameterization import EdgeWiseDistributionByModel
import pickle as pkl
import numpy as np

In [6]:
with open("result/crosswords/experiment1_final_utilities.pkl", "rb") as file:
    utilities = pkl.load(file)

In [11]:
print(utilities)
print(np.mean(utilities))

[0.8, 0.8, 0.7, 1.0, 0.7, 1.0, 1.0, 0.5, 0.5, 0.8, 0.7, 0.4, 0.4, 0.6, 0.1, 0.8, 0.5, 1.0, 0.6, 0.2]
0.655


In [23]:
potential_connections = [0 for _ in range(188)]
init_connection_probability = 0.1
domain = "crosswords"
llm_backbone_name="gpt2"

In [ ]:
swarm = Swarm(["CrosswordsReflection", "CrosswordsToT", "CrosswordsBruteForceOpt"], "crosswords", "gpt-3.5-turbo-1106", #"gpt-4-1106-preview"
            final_node_class="ReturnAll", 
            final_node_kwargs={},
            edge_optimize=True,
            init_connection_probability=init_connection_probability, 
            connect_output_nodes_to_final_node=connect_output_nodes_to_final_node, 
            include_inner_agent_connections=include_inner_agent_connections,
            edge_network_enable=edge_network_enable,
            llm_backbone_name=llm_backbone_name)

In [25]:
edge_network = EdgeNetwork(llm_backbone_name=llm_backbone_name, num_edges=len(potential_connections), initial_probability=init_connection_probability)
connection_dist = EdgeWiseDistributionByModel(potential_connections, edge_network, domain)

connection_dist.load_state_dict(torch.load("result/crosswords/experiment_edge_logits_10.pt"))

TypeError: 'int' object is not iterable

In [ ]:
# Create an instance of EdgeNetwork
num_edges = 188  # Replace with the number of edges used when saving the model
llm_backbone_name = 'gpt2'  # Replace with the name of the model used when saving the model
model = EdgeNetwork(llm_backbone_name, num_edges)

# Load the state dictionary
state_dict = torch.load('result/crosswords/experiment_edge_logits_10.pt')

# Remove 'model.' prefix from state dictionary keys and exclude 'order_params'
state_dict = {k.replace('model.', ''): v for k, v in state_dict.items() if k != 'model.order_params'}

model.load_state_dict(state_dict)

In [ ]:
# Create an instance of EdgeNetwork
num_edges = 188  # Replace with the number of edges used when saving the model
llm_backbone_name = 'gpt2'  # Replace with the name of the model used when saving the model
model = EdgeNetwork(llm_backbone_name, num_edges)

# Load the state dictionary
state_dict = torch.load('result/crosswords/experiment_edge_logits_10.pt')

# Remove 'model.' prefix from state dictionary keys
state_dict = {k.replace('model.', ''): v for k, v in state_dict.items()}

model.load_state_dict(state_dict)